# Sentiment Analysis

In [ ]:
from zst_reader import read_lines_zst, write_line_zst
import json

In [ ]:
subreddits = ["Conservative", "progressive",
              "democrats", "Republican",
              "NeutralPolitics", "PoliticalDiscussion", "politics"]

In [ ]:
input_submissions = [f"data/{s}/{s}_submissions_classified.zst" for s in subreddits]
input_sentiments = [f"analysis/1d/{s}_comments_sentiment.zst" for s in subreddits]

In [11]:
input_sentiments = [f"analysis/1d/{s}_submissions_overall_sentiment.zst" for s in subreddits]

In [ ]:
# Load the input_submissions data into a dictionary
submissions = {}
for subreddit, input_path in zip(subreddits, input_submissions):
    submissions[subreddit] = {}
    for line, file_bytes_processed in read_lines_zst(input_path):
        obj = json.loads(line)

        # Keep topic_dist, sentiment, and created_utc, num_comments

In [19]:
output_overall_sentiment = [f"data/{s}/{s}_comments_sentiment.zst" for s in subreddits]

In [21]:
# Read one sample line from input_submissions
sub_sample = output_overall_sentiment[0]

# Read one line
count = 0
for line, file_bytes_processed in read_lines_zst(sub_sample):
    obj = json.loads(line)
    print(obj)
    count += 1
    if count > 10: break

{'author': 'Fetch_1', 'created_utc': 1546329834, 'score': 8, 'ups': '', 'downs': '', 'controversiality': 1, 'gilded': 0, 'sentiment': -0.03736263736263734}
{'author': 'Fetch_1', 'created_utc': 1546329879, 'score': 6, 'ups': '', 'downs': '', 'controversiality': 0, 'gilded': 0, 'sentiment': 0.09583333333333333}
{'author': 'Milleroski', 'created_utc': 1546330675, 'score': 163, 'ups': '', 'downs': '', 'controversiality': 0, 'gilded': 0, 'sentiment': 0.0}
{'author': 'Ung-Tik', 'created_utc': 1546331607, 'score': 25, 'ups': '', 'downs': '', 'controversiality': 0, 'gilded': 0, 'sentiment': 0.14285714285714285}
{'author': 'steveoneill', 'created_utc': 1546331692, 'score': 8, 'ups': '', 'downs': '', 'controversiality': 0, 'gilded': 0, 'sentiment': 0.0}
{'author': 'PoliticalJunkDrawer', 'created_utc': 1546332165, 'score': 6, 'ups': '', 'downs': '', 'controversiality': 0, 'gilded': 0, 'sentiment': 0.02333333333333336}
{'author': 'superdmp', 'created_utc': 1546332298, 'score': 104, 'ups': '', 'dow

In [17]:
for k in obj.keys():
    print(k)

downs
ups
title
num_comments
created_utc
selftext
score
topic_dist


In [22]:
input_comments = [f"data/{s}/{s}_comments_clean.zst" for s in subreddits]
count = 0
for line, file_bytes_processed in read_lines_zst(input_comments[0]):
    obj = json.loads(line)
    print(obj)
    count += 1
    if count > 10: break

{'author': 'Fetch_1', 'created_utc': 1546329834, 'body': '1 People will die for not being able to afford healthcare The ACA as a law helps me to afford health insurance Its on the administration and congress to uphold the law of the land and not to directly sabotage legislation that helps millions of people get and afford health care 2 Climate is real and we are already seeing some effects of it Natural disasters are going to get worst and thats a fact and that we need to reduce our consumption of green house gasses Also investing and promoting different energy solutions and giving tax breaks to companies that do those things seems like smart legislation Innovation is where American thrives Have us perfect wind solar geothermal and nuclear energy and then sell the technology 4 Response to Hurricane Maria was bad Really bad the fact that the island didnt have power or medical supplies to aid those affected caused a lot of death', 'score': 8, 'ups': '', 'downs': '', 'controversiality': 1

In [23]:
input_comments = [f"data/{s}/{s}_comments_subset.zst" for s in subreddits]
count = 0
for line, file_bytes_processed in read_lines_zst(input_comments[0]):
    obj = json.loads(line)
    print(obj)
    count += 1
    if count > 10: break

{'link_id': 't3_abegnh', 'author': 'Fetch_1', 'created_utc': 1546329834, 'body': "1. People will die for not being able to afford healthcare. The ACA as a law helps me to afford health insurance. Its on the administration and congress to uphold the law of the land and not to directly sabotage legislation that helps millions of people get and afford health care.\n2. Climate is real and we are already seeing some effects of it. Natural disasters are going to get worst and that's a fact and that we need to reduce our consumption of green house gasses. Also investing and promoting different energy solutions and giving tax breaks to companies that do those things seems like smart legislation. Innovation is where American thrives. Have us perfect wind, solar, geothermal, and nuclear energy and then sell the technology.\n4. Response to Hurricane Maria was bad. Really bad, the fact that the island didnt have power or medical supplies to aid those affected caused a lot of death.", 'score': 8, '

In [14]:
sub_sample

'analysis/1d/Conservative_submissions_overall_sentiment.zst'

In [28]:
# Import library that contains TextBlob
from textblob import TextBlob
import json, zstandard

In [33]:
# Function to add the sentiment of the comments in a submission
def add_sentiment_comments(input_comments: list, output_comments: list) ->  None:
    """Add the sentiment of the comments in a submission"""

    # Loop through input paths
    for in_comment, out_comment in zip(input_comments, output_comments):

        # Create the zst handler
        handle = zstandard.ZstdCompressor().stream_writer(open(out_comment, 'wb'))

        # Save the data to zst file
        with open(out_comment, mode="w", newline="") as file:

            for line, file_bytes_processed in read_lines_zst(in_comment):
                obj = json.loads(line)

                # Skip if body doesn't exist
                if 'body' not in obj:
                    continue

                # Get body of comment
                body = obj['body']

                # Skip if body is deleted or removed
                if (body == '[deleted]') or (body == '[removed]'):
                    continue

                # Skip if score or ups+downs is too low
                if 'score' in obj:
                    try: score = int(obj['score'])
                    except: score = 0
                    if score <= 5: continue
                else:
                    ups = obj.get('ups',0)
                    try: ups = int(ups)
                    except: ups = 0
                    downs = obj.get('downs',0)
                    try: downs = int(downs)
                    except: downs = 0
                    if (ups-downs) <= 5: continue

                # Get sentiment of body
                sentiment = TextBlob(body).sentiment.polarity

                # Add sentiment to object
                obj['sentiment'] = sentiment

                # Remove body
                del obj['body']

                # If link_id is missing, skip
                if 'link_id' not in obj:
                    continue

                # From link_id, keep only the characters after the underscore
                obj['link_id'] = obj['link_id'].split('_')[-1]

                # Write the data to the zst file
                new_line = json.dumps(obj)
                write_line_zst(handle, new_line)
    
    return

In [35]:
# Create input and output paths
input_comments = [f"data/{s}/{s}_comments_clean.zst" for s in subreddits]
output_comments = [f"analysis/sentiment/{s}_comments_sentiment.zst" for s in subreddits]

# Add sentiment to comments
add_sentiment_comments(input_comments, output_comments)


In [40]:
output_submissions = [f"data/{s}/{s}_submissions_clean.zst" for s in subreddits] # [f"data/{s}/{s}_submissions_classified.zst" for s in subreddits]

In [41]:
# Print one line from the output_submissions
sub_sample = output_submissions[0]
count = 0
for line, file_bytes_processed in read_lines_zst(sub_sample):
    obj = json.loads(line)
    print(obj)
    count += 1
    if count > 10: break

{'id': 'acr5wl', 'author': 'NakedAndBehindYou', 'downs': '', 'ups': '', 'title': 'Trump is right He can declare a national emergency to build the wall and it is 100 legal and already authorized', 'num_comments': 123, 'created_utc': 1546665929, 'selftext': 'The details 1 Decades ago Congress gave the Secretary of Defense the power to use military construction funds to build anything he wants if the President first declares a national emergency 2 The President can declare a national emergency for any reason 3 The 2019 budget for military construction is 103B This budget is already funded by Congress Now the military is probably already planning on using those funds for other purposes But if Trump really wanted to he could declare a national emergency and direct the military to spend all 103B of it on building his wall Trump has thus already won the fight over the wall funding but is just negotiating with Democrats because he doesnt want to screw over the military by taking away their con

### Add sentiment to comments

In [70]:
# Load NRC emotion lexicon from NRC-Emotion.txt
nrc = {}
with open('NRC-Emotion.txt', 'r') as f:
    for line in f:
        words, emotion, value = line.strip().split('\t')

        # Replace - with spaces
        words = words.replace('-', ' ').replace(",", ' ').replace("  ", " ")

        # Break words into a list of words
        words = words.split(' ')

        for word in words:
            if word not in nrc.keys():
                nrc[word] = {}
            nrc[word][emotion] = int(value)

# Save nrc to a json file as lexicon_clean.json
with open('lexicon_clean.json', 'w') as f:
    json.dump(nrc, f)

In [54]:
# Import library that contains TextBlob
from textblob import TextBlob


In [85]:
# Function to add the sentiment of the comments in a submission
def add_sentiment_comments(input_comments: list, output_comments: list, lexicon: dict) ->  None:
    """Add the sentiment of the comments in a submission"""

    # Loop through input paths
    for in_comment, out_comment in zip(input_comments, output_comments):

        # Create the zst handler
        handle = zstandard.ZstdCompressor().stream_writer(open(out_comment, 'wb'))

        # Save the data to zst file
        with open(out_comment, mode="w", newline="") as file:

            for line, file_bytes_processed in read_lines_zst(in_comment):
                obj = json.loads(line)

                # Skip if body doesn't exist
                if 'body' not in obj:
                    continue

                # Get body of comment
                body = obj['body']

                # Skip if body is deleted or removed
                if (body == '[deleted]') or (body == '[removed]'):
                    continue

                # Skip if score or ups+downs is too low
                if 'score' in obj:
                    try: score = int(obj['score'])
                    except: score = 0
                    if score <= 5: continue
                else:
                    ups = obj.get('ups',0)
                    try: ups = int(ups)
                    except: ups = 0
                    downs = obj.get('downs',0)
                    try: downs = int(downs)
                    except: downs = 0
                    if (ups-downs) <= 5: continue

                # Get polarity and subjectivity of body
                polarity = TextBlob(body).sentiment.polarity
                subjectivity = TextBlob(body).sentiment.subjectivity

                # Calculate emotions in the comment using NRC emotion lexicon
                emotions = {'fear': 0, 'anger': 0, 'anticip': 0, 'trust': 0,
                            'surprise': 0, 'sadness': 0, 'joy': 0, 'disgust': 0,
                            'positive': 0, 'negative': 0}
                for word in body.split():
                    if word in nrc_lexicon.keys():
                        for emotion in nrc_lexicon[word].keys():
                            emotions[emotion] = emotions.get(emotion, 0) + nrc_lexicon[word][emotion]

                # Add sentiment and emotion to object
                obj['polarity'] = polarity
                obj['subjectivity'] = subjectivity
                obj['emotions'] = emotions

                # Remove body
                del obj['body']

                # Write the data to the zst file
                new_line = json.dumps(obj)

                write_line_zst(handle, new_line)
    
    return

In [83]:
# Load nrc_lexicon
nrc_lexicon = json.load(open('lexicon_clean.json', 'r'))

In [86]:
# Create input and output paths
input_comments = [f"data/{s}/{s}_comments_clean.zst" for s in subreddits]
output_comments = [f"data/{s}/{s}_comments_sentiment.zst" for s in subreddits]

# Add sentiment to comments
add_sentiment_comments(input_comments, output_comments, nrc_lexicon)
